# Linear Equations


## Matrix-Matrix Multiplication

Before diving into methods for solving linear equations, we need to define a new operation the matrix-matrix multiplication. This operation takes two matrices and produces a third matrix. Given a matrix $\mathbf{A}$ of size $m \times n$ and a matrix $\mathbf{B}$ of size $n \times p$, their product $\mathbf{C} = \mathbf{A}\mathbf{B}$ is a matrix of size $m \times p$. The element in the i-th row and j-th column of $\mathbf{C}$, denoted as $c_{ij}$, is computed as the dot product of the i-th row of $\mathbf{A}$ and the j-th column of $\mathbf{B}$:
$$c_{ij} = \sum_{k=1}^{n} a_{ik} b_{kj}$$

This can be implemented in Python as follows:

In [1]:
import numpy as np

def matrix_matrix_multiply(A, B):
    """Multiplies matrix A by a matrix B."""

    m = A.shape[0]
    n = A.shape[1]
    p = B.shape[1]
    assert n == B.shape[0], "Incompatible dimensions for matrix multiplication."

    C = np.zeros((m, p))

    for i in range(m):
        for j in range(p):
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]

    return C

A = np.array([[1, 2, 3],
              [4, 5, 6]])
B = np.array([[7, 8],
              [9, 10],
              [11, 12]])

C = matrix_matrix_multiply(A, B)
print("Result of A @ B:")
print(C)
print(A @ B)  # Verify with NumPy's built-in function


Result of A @ B:
[[ 58.  64.]
 [139. 154.]]
[[ 58  64]
 [139 154]]


Note that you can use NumPy's built-in function `np.dot()` or the `@` operator for matrix multiplication, which are optimized for performance.

## Gaussian Elimination

From the last chapter, we know that a system of linear equations can be represented in matrix form as: $\mathbf{A}\mathbf{x} = \mathbf{b}$.

In a system of two equations with two variables, this can be expressed as:

$$\begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} = \begin{bmatrix} b_1 \\ b_2 \end{bmatrix}$$
$$\begin{cases} a_{11}x_1 + a_{12}x_2 = b_1 \\ a_{21}x_1 + a_{22}x_2 = b_2 \end{cases}$$

Which you probably already know how to solve by substitution or elimination. However, as the number of equations and variables increases, it becomes impractical to solve these systems by hand. Instead, we can develop algorithms to solve them more efficiently.

Start by eliminating the first variable, $x_1$, from the second equation. To do this, we can multiply the first equation by $\frac{a_{21}}{a_{11}}$ and subtract it from the second equation:

$$\begin{cases} a_{11}x_1 + a_{12}x_2 = b_1 \\ \left(a_{21} - \frac{a_{21}}{a_{11}}a_{11}\right)x_1 + \left(a_{22} - \frac{a_{21}}{a_{11}}a_{12}\right)x_2 = b_2 - \frac{a_{21}}{a_{11}}b_1 \end{cases}$$

This simplifies to:

$$\begin{cases} a_{11}x_1 + a_{12}x_2 = b_1 \\ \left(a_{22} - \frac{a_{21}a_{12}}{a_{11}}\right)x_2 = b_2 - \frac{a_{21}}{a_{11}}b_1 \end{cases}$$

At this point, we can solve for $x_2$ directly from the second equation and then substitute back to find $x_1$. Then we have, $x_2 = \frac{b_2 - \frac{a_{21}}{a_{11}}b_1}{a_{22} - \frac{a_{21}a_{12}}{a_{11}}}$ and $x_1 = b_1 - a_{12}x_2$.

This can be repeated for a larger system of equations, by systematically eliminating variables from the equations below them, a process known as Gaussian elimination. Once in upper triangular form, we can then perform back substitution to find the values of the variables. In general for an $n \times n$ system, the algorithm can be summarized as follows:

1. For each row $i$ from 1 to $n-1$:
   - For each row $j$ from $i+1$ to $n$:
     - Compute the multiplier $m = \frac{a_{ji}}{a_{ii}}$
     - Update row $j$: 
       - $a_{jk} = a_{jk} - m \cdot a_{ik}$ for all columns $k$ from $i$ to $n$
       - $b_j = b_j - m \cdot b_i$
2. After obtaining the upper triangular matrix, perform back substitution:
    - For each row $i$ from $n$ down to 1:
      - Compute $x_i = \frac{b_i - \sum_{j=i+1}^{n} a_{ij} x_j}{a_{ii}}$

This method can be implemented in Python as:

In [2]:
def gaussian_elimination(A, b):
    """Solves the system of linear equations Ax = b using Gaussian elimination.

    Args:
        A (np.ndarray): Coefficient matrix (n x n).
        b (np.ndarray): Right-hand side vector (n,).
    Returns:
        x (np.ndarray): Solution vector (n,).
    """
    n = len(b)
    # Convert to float for division
    A = A.astype(float)
    b = b.astype(float)
    # Forward elimination
    for i in range(n - 1): # for each pivot row
        for j in range(i + 1, n): # for each row below pivot
            multiplier = A[j, i] / A[i, i]
            A[j, i:] -= multiplier * A[i, i:]
            b[j] -= multiplier * b[i]
    # Back substitution
    x = np.zeros(n)
    for i in range(n - 1, -1, -1): # from last row to first
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x

# Example usage
A = np.array([[1, 2, 3],
              [2, 5, 2],
              [6, -3, 1]])
b = np.array([8, -11, -3])
x = gaussian_elimination(A, b)
print("Solution:", x)
print("Verification (Ax):", A @ x, "should equal b:", b)
print("Close to Numpy solution:", np.allclose(x, np.linalg.solve(A, b)))


Solution: [-3.14285714 -3.31168831  5.92207792]
Verification (Ax): [  8. -11.  -3.] should equal b: [  8 -11  -3]
Close to Numpy solution: True


Often in practical applications, we need to solve multiple systems of equations with the same coefficient matrix $\mathbf{A}$ but different right-hand side vectors $\mathbf{b}$. In such cases, it is efficient to perform the LU decomposition of $\mathbf{A}$ once and then use it to solve for each $\mathbf{b}$. The LU decomposition factors the matrix $\mathbf{A}$ into a lower triangular matrix $\mathbf{L}$ and an upper triangular matrix $\mathbf{U}$ such that $\mathbf{A} = \mathbf{L}\mathbf{U}$. Then, for each right-hand side vector $\mathbf{b}$, we can solve the two triangular systems $\mathbf{L}\mathbf{y} = \mathbf{b}$ and $\mathbf{U}\mathbf{x} = \mathbf{y}$ using forward and backward substitution, respectively. This approach significantly reduces computational effort when solving multiple systems with the same coefficient matrix.

The U matrix we already have from Gaussian elimination. To get the L matrix, we can keep track of the multipliers used during the elimination process. The multipliers form the entries of the L matrix below the main diagonal, while the diagonal entries of L are all 1s.


In [3]:
def LU_decomposition(A):
    n = A.shape[0]
    L = np.eye(n)
    U = A.astype(float)
    for i in range(n - 1):
        for j in range(i + 1, n):
            multiplier = U[j, i] / U[i, i]
            L[j, i] = multiplier
            U[j, i:] -= multiplier * U[i, i:]
    return L, U

def solve_LU(L, U, b):
    n = len(b)
    # Forward substitution to solve Ly = b
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    # Back substitution to solve Ux = y
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

# Example usage of LU decomposition
A = np.array([[1, 2, 3],
              [2, 5, 2],
              [6, -3, 1]])
b = np.array([8, -11, -3])
L, U = LU_decomposition(A)
print("L matrix:\n", L)
print("U matrix:\n", U)
print("Verification (LU):\n", L @ U, "should equal A:\n", A)
x = solve_LU(L, U, b)
print("Solution using LU:", x)
print("Close to Numpy solution:", np.allclose(x, np.linalg.solve(A, b)))


L matrix:
 [[  1.   0.   0.]
 [  2.   1.   0.]
 [  6. -15.   1.]]
U matrix:
 [[  1.   2.   3.]
 [  0.   1.  -4.]
 [  0.   0. -77.]]
Verification (LU):
 [[ 1.  2.  3.]
 [ 2.  5.  2.]
 [ 6. -3.  1.]] should equal A:
 [[ 1  2  3]
 [ 2  5  2]
 [ 6 -3  1]]
Solution using LU: [-3.14285714 -3.31168831  5.92207792]
Close to Numpy solution: True


This algorithm works well for almost all systems of linear equations, except in cases where one of the pivot elements (the diagonal elements during elimination) is zero. For example, consider the system:
$$\begin{bmatrix} 0 & 2 \\ 1 & 3 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} = \begin{bmatrix} 4 \\ 5 \end{bmatrix}$$

The first pivot is zero, which will give a division by zero error during elimination. To handle such cases, we can use partial pivoting, which involves swapping rows to ensure that the pivot element is non-zero. We will be using the $\mathbf{PA} = \mathbf{LU}$ where $\mathbf{P}$ is a permutation matrix that accounts for the row swaps. A permutation matrix is obtained by permuting the rows of an identity matrix. For example, swapping the first and second rows of a $2 \times 2$ identity matrix gives:
$$\mathbf{P} = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$$

Also, for numerical stability, it is common to choose the largest available pivot element in absolute value from the column being considered. This helps to minimize rounding errors during the elimination process.

Here's how we can implement Gaussian elimination with partial pivoting in Python:


In [4]:
def lu_decomposition_with_pivoting(A):
    n = A.shape[0]
    L = np.eye(n)
    U = A.astype(float)
    P = np.eye(n)
    for i in range(n - 1):
        # Pivoting
        max_row_index = np.argmax(np.abs(U[i:, i])) + i
        if max_row_index != i:
            U[[i, max_row_index], :] = U[[max_row_index, i], :]
            P[[i, max_row_index], :] = P[[max_row_index, i], :]
            if i > 0:
                L[[i, max_row_index], :i] = L[[max_row_index, i], :i]
        for j in range(i + 1, n):
            multiplier = U[j, i] / U[i, i]
            L[j, i] = multiplier
            U[j, i:] -= multiplier * U[i, i:]
    return P, L, U

def solve_LU_with_pivoting(P, L, U, b):
    # Apply permutation to b
    b = P @ b
    n = len(b)
    # Forward substitution to solve Ly = b
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    # Back substitution to solve Ux = y
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

# Example usage of LU decomposition with pivoting
A = np.array([[1, 2, 3],
              [2, 5, 2],
              [6, -3, 1]])
b = np.array([8, -11, -3])
P, L, U = lu_decomposition_with_pivoting(A)
print("P matrix:\n", P)
print("L matrix:\n", L)
print("U matrix:\n", U)
print("Verification (LU):\n", P @ A, "should equal L @ U:\n", L @ U)
x = solve_LU_with_pivoting(P, L, U, b)
print("Solution using LU with pivoting:", x)
print("Close to Numpy solution:", np.allclose(x, np.linalg.solve(A, b)))


P matrix:
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
L matrix:
 [[1.         0.         0.        ]
 [0.33333333 1.         0.        ]
 [0.16666667 0.41666667 1.        ]]
U matrix:
 [[ 6.         -3.          1.        ]
 [ 0.          6.          1.66666667]
 [ 0.          0.          2.13888889]]
Verification (LU):
 [[ 6. -3.  1.]
 [ 2.  5.  2.]
 [ 1.  2.  3.]] should equal L @ U:
 [[ 6. -3.  1.]
 [ 2.  5.  2.]
 [ 1.  2.  3.]]
Solution using LU with pivoting: [-3.14285714 -3.31168831  5.92207792]
Close to Numpy solution: True


## Matrix Inverses

Not used often in numerical methods, but still important to understand is the matrix inverse. The inverse of a square matrix $\mathbf{A}$, denoted as $\mathbf{A}^{-1}$, is defined such that:
$$\mathbf{A}\mathbf{A}^{-1} = \mathbf{A}^{-1}\mathbf{A} = \mathbf{I}$$

Matrix inverses can also be used to solve systems of linear equations. If $\mathbf{A}$ is invertible, the solution to the system $\mathbf{A}\mathbf{x} = \mathbf{b}$ can be found by multiplying both sides by $\mathbf{A}^{-1}$:
$$\mathbf{A}^{-1}\mathbf{A}\mathbf{x} = \mathbf{A}^{-1}\mathbf{b}$$
$$\mathbf{x} = \mathbf{A}^{-1}\mathbf{b}$$

To compute the inverse of a matrix using Gaussian elimination, we can augment the matrix $\mathbf{A}$ with the identity matrix $\mathbf{I}$ and perform row operations to transform $\mathbf{A}$ into $\mathbf{I}$. The augmented part will then become $\mathbf{A}^{-1}$. Here's how we can implement this in Python:


In [9]:
def matrix_inverse(A):
    n = A.shape[0]
    # Create an augmented matrix [A | I]
    A = A.astype(float)
    AI = np.hstack((A, np.eye(n)))

    for i in range(n):
        # Make the diagonal contain all 1s
        AI[i] = AI[i] / AI[i, i]

        for j in range(n):
            if i != j:
                AI[j] = AI[j] - AI[j, i] * AI[i]

    # The right half of the augmented matrix is now A^-1
    A_inv = AI[:, n:]
    return A_inv

# Example usage of matrix inverse
A = np.array([[1, 7],
              [2, 6]])
A_inv = matrix_inverse(A)
print("Inverse of A:\n", A_inv)
print("Verification (A @ A_inv):\n", A @ A_inv, "should equal identity matrix:\n", np.eye(A.shape[0]))
print("Close to Numpy inverse:", np.allclose(A_inv, np.linalg.inv(A)))


Inverse of A:
 [[-0.75   0.875]
 [ 0.25  -0.125]]
Verification (A @ A_inv):
 [[1. 0.]
 [0. 1.]] should equal identity matrix:
 [[1. 0.]
 [0. 1.]]
Close to Numpy inverse: True


## Linear Equations

Now that we know how to solve linear equations we can proceed to discuss the different types of solutions that can arise when solving systems of linear equations.